In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

/Users/martinmaele/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/martinmaele/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [7]:
# Example: Load some dataset 
ais_train = pd.read_csv('ais_train.csv',sep='|') 
ais_test = pd.read_csv('ais_test.csv',sep=',')

# Drop both the latitude and longitude columns from the training data
X_train = ais_train[['vesselId','time']]
X_train['time'] = pd.to_datetime(X_train['time']) 
X_test = ais_test[['vesselId','time']]
X_test['time'] = pd.to_datetime(X_test['time'])
y_train = ais_train[['latitude','longitude']] 



/var/folders/8g/d2wk3x0d7jz2_0srl03y6_mm0000gn/T/ipykernel_1994/847195497.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['time'] = pd.to_datetime(X_train['time'])
/var/folders/8g/d2wk3x0d7jz2_0srl03y6_mm0000gn/T/ipykernel_1994/847195497.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['time'] = pd.to_datetime(X_test['time'])


In [10]:
# Identify feature types. I also have a datatime feature. WHat to do with that one? 
X_train['year'] = X_train['time'].dt.year
X_train['month'] = X_train['time'].dt.month
X_train['day'] = X_train['time'].dt.day
X_train['hour'] = X_train['time'].dt.hour
X_train['minute'] = X_train['time'].dt.minute
X_train['second'] = X_train['time'].dt.second
X_train['day_of_week'] = X_train['time'].dt.dayofweek
X_train['day_of_year'] = X_train['time'].dt.dayofyear
X_train['is_weekend'] = X_train['time'].dt.dayofweek >= 5 

# Same fore the test set
X_test['year'] = X_test['time'].dt.year
X_test['month'] = X_test['time'].dt.month
X_test['day'] = X_test['time'].dt.day
X_test['hour'] = X_test['time'].dt.hour
X_test['minute'] = X_test['time'].dt.minute
X_test['second'] = X_test['time'].dt.second
X_test['day_of_week'] = X_test['time'].dt.dayofweek
X_test['day_of_year'] = X_test['time'].dt.dayofyear
X_test['is_weekend'] = X_test['time'].dt.dayofweek >= 5

# Drop the datetime feature
X_train = X_train.drop(columns=['time'])
X_test = X_test.drop(columns=['time'])

# define the feature types 
categorical_features = ['vesselId','is_weekend']
numerical_features = ['year','month','day','hour','minute','second','day_of_week','day_of_year']

# Define transformers 
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Combine transformers in a ColumnTransformer 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

/var/folders/8g/d2wk3x0d7jz2_0srl03y6_mm0000gn/T/ipykernel_1994/996769074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['year'] = X_train['time'].dt.year
/var/folders/8g/d2wk3x0d7jz2_0srl03y6_mm0000gn/T/ipykernel_1994/996769074.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['month'] = X_train['time'].dt.month
/var/folders/8g/d2wk3x0d7jz2_0srl03y6_mm0000gn/T/ipykernel_1994/996769074.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from xgboost import XGBRegressor
import numpy as np

# Create a pipeline that combines the preprocessor with the XGBoost classifier
# Create a pipeline that combines the preprocessor with the XGBoost regressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(objective='reg:squarederror'))
]) 

# Define the parameter grid for grid search
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [3, 5, 7],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__subsample': [0.8, 1.0]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation MSE: {:.2f}".format(-grid_search.best_score_))

# Fit the final model with the best parameters on the entire training set
final_model = grid_search.best_estimator_ 
final_model.fit(X_train, y_train) 

/Users/martinmaele/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/martinmaele/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/martinmaele/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/martinmaele/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' cur

Best parameters found:  {'regressor__learning_rate': 0.2, 'regressor__max_depth': 7, 'regressor__n_estimators': 200, 'regressor__subsample': 1.0}
Best cross-validation MSE: 2122.71


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'month', 'day',
                                                   'hour', 'minute', 'second',
                                                   'day_of_week',
                                                   'day_of_year']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['vesselId',
                                                   'is_weekend'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=N...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.2,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=7, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=200, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [14]:
# I want to do a prediction on the test set
y_pred = final_model.predict(X_test)


In [18]:
y_pred = pd.DataFrame(y_pred, columns=['latitude','longitude'])
y_pred.to_csv('ais_test_pred_førsteFaenskap.csv', index=False)

prediksjon = pd.read_csv('ais_test_pred_førsteFaenskap.csv')

print(prediksjon.to_string())

# Oversikt legg sammen prediksjon og test settet.



        latitude   longitude
0      24.274570   -0.341592
1      34.874924  101.140450
2      34.874924    8.586983
3     -29.706644  150.518400
4      43.380436    6.695461
5      46.464207    6.695461
6      38.330350    6.695461
7      45.847590    3.293982
8      37.709324    6.695461
9      37.709324    6.695461
10     37.709324    6.695461
11     42.068490    6.695461
12     49.362907    6.695461
13     40.409260    6.695461
14     37.709324    1.201120
15     50.796406    6.695461
16    -13.297948  128.098070
17     37.709324    6.127533
18     40.900370    6.695461
19     50.842740    6.695461
20     48.887230    6.695461
21     37.709324    6.215744
22     19.991812   31.214802
23     37.709324  -60.296886
24     37.709324   -0.690320
25     37.709324    6.695461
26     31.347698  -91.627750
27     39.323240    6.695461
28     50.537598    6.695461
29     33.236977    6.695461
30     35.018402    6.695461
31     49.462890    6.695461
32     40.409010    6.695461
33     40.8460